# SEP Forecasting 
***

Current physics based models of SEPs are unable to execute sufficiently fast in order to
provide actionable information towards forecasting such disturbances, which can impact
Earth within tens of minutes of the onset of an eruptive event. This is compounded by
the intrinsic latency of certain key observations, which are used to define the initial
conditions of these models. Instead, there is a reliance on statistical models to provide
forecast probabilities of Earth-bound SEPs using real-time data. Since the largest, most
impactful events occur infrequently, some regions of the feature space are sparse and
simple discrete binning procedures have limitations. The goal of this project is to
improve upon the empirical SEP proton prediction forecast model (PROTONS) currently
in operational use at SWPC, through the application of modern machine learning
techniques.

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

from datetime import datetime,timedelta

from pandas.plotting import scatter_matrix

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import shuffle

from scipy.sparse import csr_matrix, find


#### Reading in the SEP and control data

In [2]:
#read in the Balch SEP event list
sepdata = pd.read_excel("SPEall.v7p.xls")

#read in the Balch SEP event list
ctrldata = pd.read_excel("ctrlevents.v8p.xls")

In [22]:
list(sepdata)

['Onset',
 'Threshold',
 'Maxtime',
 'Maxflux',
 'Endtime',
 'EPSSatId',
 'FlrOnset',
 'Flrmaxtime',
 'Flrendtime',
 'FlrPeakFlux',
 'xrscls',
 'ocls',
 'optlocation',
 'region',
 'TypeII',
 'TII_Duration',
 'TypeIV',
 'TIV_duration',
 'CME_speed',
 'XRSSatId',
 'FlrHpTime',
 'FlrIntFlux',
 'FlrIntFlux2',
 'P10MAX',
 'P30MAX',
 'P60MAX',
 'P100MAX',
 'P10FLUENCE',
 'P30FLUENCE',
 'P60FLUENCE',
 'P100FLUENCE',
 'SEP']

In [23]:
list(ctrldata)

['FlrOnset',
 'Flrmaxtime',
 'Flrendtime',
 'FlrPeakFlux',
 'xrscls',
 'ocls',
 'optlocation',
 'region',
 'Rpatrol',
 'Type II',
 'TII Duration',
 'Type IV',
 'TIV duration',
 'CME Patrol',
 'CME onset',
 'CME speed',
 'Satellite',
 'FlrHpTime',
 'FlrIntFlux',
 'FlrIntFlux2',
 'Association',
 'S1onset',
 'S2onset',
 'S3onset',
 'S4onset',
 'S5onset',
 'GT100onset',
 'SEP']

<span style="color:red"> **Gah, the sepdata and ctrldata dataframes DO NOT have the same columns. At some point we'll need to make a master list with both sep and "all clear" events, but for right now, just to get the feature engineering pipeline up and running, I'm just going to label half of the sep events as being all clear, just to give me two classes.**</span>

In [35]:
#add y label column indicating positive and negative SEP events
#sepdata['SEP'] = 1
#ctrldata['SEP'] = 0

#merge sep events and control events
#allevts = pd.concat([sepdata, ctrldata])

#shuffle the events so they are not organized 
#allevts_shuffled = shuffle(allevts)

#save the shuffled dataframe
#allevts.to_csv("AllEvtsShuffled.csv")

In [41]:
#adding y label column indicating positive and negative SEP events (HACK 50/50 pos and neg to test pipeline)
sepdata['SEP'] = 0
sepdata.SEP[0:int(len(sepdata)/2)] = 1

#shuffle the events so they are not organized 
allevts_shuffled = shuffle(sepdata)

#save the shuffled dataframe
allevts_shuffled.to_csv("AllEvtsShuffled.csv")

/Users/hazelbain/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### Make Scatterplot matrix showing relationship between parameters

In [32]:
#pd.plotting.scatter_matrix(sepdata[['Maxflux','FlrPeakFlux','TII_Duration','TIV_duration',\
#                            'FlrIntFlux','FlrIntFlux2','P10MAX','P30MAX','P60MAX',\
#                            'P100MAX','P10FLUENCE','P30FLUENCE','P60FLUENCE','P100FLUENCE']], \
#                            figsize=(18,18), diagonal='kde')

### Logisitic Regression

#### Class to exract features from dataframe that can be used directly

In [7]:
class RawFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        return None
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""
        
        return ["FlTim2Pk"]
    
    def transform(self, examples):
        
        # Initiaize matrix 
        X = np.zeros((len(examples),1))           
        
        ## TODO
        
        #time between flare max and flare onset
        #X[ii,:] = examples.Flrmaxtime - examples.FlrOnset
        
        return(X)

#### Class to create feature with the time between flare onset to flare max time

In [118]:

class FlareTime2Peak(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        return None
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""
        
        return ["FlTim2Pk"]
    
    def transform(self, examples):
        
        # Initiaize matrix 
        X = np.zeros((len(examples),1))           

        #time between flare max and flare onset
        X[:,0] = np.asarray([(mx - on).seconds for mx, on in zip(examples.Flrmaxtime, examples.Onset)])
        
        return(X)

In [119]:
class SEPClass:
    def __init__(self):        # <--- other keywords to be used by Feature Union go here
        
        #Set up the Feature union to combine Feature creating classes
        self.allmyfeatures = FeatureUnion([
            ("RawFeat", RawFeatures()),
            ("FlareTime2Peak", FlareTime2Peak())   #,
            ])
    
    def build_train_features(self, examples):
        """
        Method to take in training text features and do further feature engineering 
        Most of the work in this homework will go here, or in similar functions  
        :param examples: currently just a list of forum posts  
        """  
        
        ##convert columns of time from string to datetime -- MOVE TO SEPARATE FUNCTION
        examples.Onset = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.Onset)
        examples.Flrmaxtime = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.Flrmaxtime)
        examples.Flrendtime = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.Flrendtime)
        examples.FlrHpTime = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.FlrHpTime)
        
        return self.allmyfeatures.fit_transform(examples)
        

    def get_test_features(self, examples):
        """
        Method to take in test text features and transform the same way as train features 
        :param examples: currently just a list of forum posts  
        """
    
        return self.allmyfeatures.transform(examples)

    def show_topX(self, num=3):
        """
        prints the top num features for the positive class and the 
        top 10 features for the negative class. 
        """
        feature_names = np.asarray([x.split("__")[1] for x in self.allmyfeatures.get_feature_names()])
        topX = np.argsort(self.logreg.coef_[0])[-num:]
        bottomX = np.argsort(self.logreg.coef_[0])[:num]
        
        print("\nTop 3 features for Pos and Neg\n-------------------------")
        for fn in np.arange(1,num):
            print("Pos %i: %s %f" % (fn, feature_names[topX[-fn]], self.logreg.coef_[0,topX[-fn]]))
        for fn in np.arange(0,num-1):
            print("Neg %i: %s %f" % (fn, feature_names[bottomX[fn]], self.logreg.coef_[0,bottomX[fn]]))

      
    def show_misclassified(self):     

        """
        Method to show the misclassified examples i.e. False Positives and False negatives 
        """
        
        #get all the feature names
        words = feat.allmyfeatures.get_feature_names()
        
        # False positives
        print("\nSome misclassified examples:")
        falsepos = np.where((self.train_pred != self.y_train) & (self.train_pred == 1))[0]   #all false pos example rows
        print("\nPredicted SEP but labeled AllClear (False Pos) \n------------------------- ")

        for i in range(len(falsepos[0:10])):         #loop through falsepos examples
            weights_falsepos = []
            x = find(feat.X_train[falsepos[i]])      #find which features are used for this example
            for ii in x[1]:                          #from sparse matrix get column indices corresponding to features
                weights_falsepos.append((words[ii].split('__')[1], self.logreg.coef_[0,ii]))      #get the word and weight

            print("label: %i, prediction %i, Neg Prob: %f, Pos Prob: %f, Ex No.: %i,  example: %s " % \
                (self.y_train[falsepos[i]], self.train_pred[falsepos[i]] , self.train_pred_prob[falsepos[i]][0], \
                     self.train_pred_prob[falsepos[i]][1], falsepos[i], self.clean_examples[falsepos[i]]))
            for j in weights_falsepos:
                print(j)
                
        # False Negatives
        falseneg = np.where((self.train_pred != self.y_train) & (self.train_pred == 0))[0]
        print("\nPredicted AllClear but labeled SEP (False Neg) \n-------------------------")

        for i in range(len(falseneg[0:10])):
            weights_falseneg = []
            x = find(feat.X_train[falseneg[r]])
            for ii in x[1]:
                weights_falseneg.append((words[ii].split('__')[1], self.logreg.coef_[0,ii])) 

            print("label: %i, prediction %i, Neg Prob: %f, Pos Prob: %f, Ex No.: %i, \nexample: %s " % \
                (self.y_train[falseneg[i]], self.train_pred[falseneg[i]], self.train_pred_prob[falseneg[i]][0], \
                 self.train_pred_prob[falseneg[i]][1], falseneg[i], self.clean_examples[falseneg[i]]))
            for j in weights_falseneg:
                print(j)
        
        print("\n\n")
       

    def score(self, predictions, y, dfTrain):

        """find the accuracy score given the labels and the predictions"""
        
        print("Accuracy: %f" % accuracy_score(y, predictions))


    def train_model(self, random_state=1234):
        """
        Method to read in training data from file, and 
        train Logistic Regression classifier. 
        
        :param random_state: seed for random number generator 
        """
        
        from sklearn.linear_model import LogisticRegression 
        
        # load data 
        self.dfTrain = pd.read_csv("AllEvtsShuffled.csv")
                      
        # get training features and labels 
        self.X_train = self.build_train_features(self.dfTrain)    #CHANGE
        self.y_train = np.array(self.dfTrain.SEP, dtype=int)
        
        #print the shape of the features
        print("Shape of the Features: Num examples x Num Features")
        print(self.X_train.shape)
        
        # train logistic regression model.  !!You MAY NOT CHANGE THIS!! 
        self.logreg = LogisticRegression(random_state=random_state)
        self.logreg.fit(self.X_train, self.y_train)

        # make predictions on training data 
        self.train_pred = self.logreg.predict(self.X_train)

        #return the LogReg probabilities used to classify each example  
        self.train_pred_prob = self.logreg.predict_proba(self.X_train)
        
        print("\nTraining Set\n-------------------------")
        self.score(self.train_pred, self.y_train, self.dfTrain)
             
        #cross validation
        folds = 2
        
        print("\nCross Validation Accuracy Scores (cross_val_score)\n-------------------------")
        scores = cross_val_score(self.logreg, self.X_train, self.y_train, cv=folds)
        print(scores)
        print("\nMean Accuracy in Cross-Validation = %.3f \n" % scores.mean())
        
        #cross validation
        print("\nCross Validation Accuracy Scores (cross_val_predict)\n-------------------------")
        self.y_pred = cross_val_predict(self.logreg, self.X_train, self.y_train, cv=folds)
        print(self.score(self.y_pred, self.y_train, self.X_train))
        
        print("\nCross Confusion Matrix\n-------------------------")
        self.conf_mat = confusion_matrix(self.y_train,self.y_pred)
        tn, fp, fn, tp = self.conf_mat.ravel()
        print("True Pos: %i, True Neg: %i, False Pos: %i,. False Neg: %i" % (tp,tn,fp,fn))
        
          
        
    def model_predict(self):
        """
        Method to read in test data from file, make predictions
        using trained model, and dump results to file 
        """
        
        # read in test data 
        dfTest  = pd.read_csv("../data/spoilers/test.csv")
        
        # featurize test data 
        self.X_test = self.get_test_features(list(dfTest["sentence"]))
        
        # make predictions on test data 
        pred = self.logreg.predict(self.X_test)
        
        # dump predictions to file for submission to Kaggle  
        pd.DataFrame({"spoiler": np.array(pred, dtype=bool)}).to_csv("prediction.csv", index=True, index_label="Id")
        
                

In [120]:
#Initialize SEPClass instance
sep = SEPClass()

#Turn off features that aren't fully fleshed out or don't work
sep.allmyfeatures.set_params(RawFeat=None)

#train the model
sep.train_model(random_state=1230)

Shape of the Features: Num examples x Num Features
(127, 1)

Training Set
-------------------------
Accuracy: 0.503937

Cross Validation Accuracy Scores (cross_val_score)
-------------------------
[ 0.5         0.50793651]

Mean Accuracy in Cross-Validation = 0.504 


Cross Validation Accuracy Scores (cross_val_predict)
-------------------------
Accuracy: 0.503937
None

Cross Confusion Matrix
-------------------------
True Pos: 0, True Neg: 64, False Pos: 0,. False Neg: 63
